# Find the DosR binding sites in bacterial genomes, and associated gene regions

- I recommend using pypy for this, as some of these analyses are computationally expensive.
- We start by reading in the data, and running a Gibbs sampler with varying ks to get an idea of how big the motifs are.
- next, we zero in on our best guess of k, which just happens to be 20 (I cheated a little and checked out Park et al. 2003).
- Once we have our motif matrix, we find a consensus string.
- And last, we look for start codons and stop codons downstream from the promotors to take a stab at finding the associated genes. 

## Import and parse the data:

 By default, the parseFile.parse() function assumes a newline delimited file with a header row of parameters. This can be changed, see the docstring for parseFile() for more info.

In [1]:
import sys, parseFile, gibbsSamplerScript, math, medianString

#Your file here:
file = "DosR.txt"
data = parseFile.parse(file)
params, dna = data
k, t = params

## Find an estimate of motif length:

 This runs a quick and dirty gibbs sampler on ks between 10 and 25, and print the best kmer for each.  By default, it uses hamming distance from consensus strings as its minimization function, but it can also use entropy. See gibbsSampler.py for info.

In [3]:



motifs = {'bestK' : 0, 'bestKScore': math.inf, 'bestMotif' : ''}
for i in range(15, 25):
    answer = gibbsSamplerScript.runGibbsSampler(dna=dna, k=i, t=t, reps=30, searchLength=3000)
    answerDict = {}
    answerDict["score"] = answer[0]
    answerDict["motifs"] = answer[1]
    answerDict["ratio"] = answerDict["score"]/i
    #answerDict["consensus"] = medianString.medianString(answerDict["motifs"], i)
    if answerDict["ratio"] < motifs["bestKScore"]:
        motifs["bestK"] = i
        motifs["bestKScore"] = answerDict["ratio"]
        #motifs["bestMotif"] = answerDict["consensus"]
    motifs[i] = answerDict
    ratio=answerDict["ratio"]
    #consensus = answerDict["consensus"]
    print(f"Per-base score for k = {i} is: {ratio}")

print("bestK:      ", motifs['bestK'])
print("bestKScore: ", motifs['bestKScore'])
#print("Best consensus motif: ", motifs["bestMotif"])
print("Printing answers:")
print(motifs)

GGGACTTCAGGCCCT GGGTCAAACGACCCT GGGACGTAAGTCCCT GACACACAAGCGCCA GGGGCGAAAGTCCCT CGGTCTTGAGGACCT GGGACTTCTGTCCCT GGGACTTTCGGCCCT AGGACTAACGGCCCT GGGACCGAAGTCCCC 35
Per-base score for k = 15 is: 2.3333333333333335
GGGACTTCAGGCCCTA GGGTCAAACGACCCTA GGGACGTAAGTCCCTA GTGGGCAGCCTCCATA GTGACCGACGTCCCCA AGGACCTTCGGCCCCA GGGACTTCTGTCCCTA GGGACTTTCGGCCCTG GGGACCAACGCCCCTG GGGACCGAAGTCCCCG 38
Per-base score for k = 16 is: 2.375
GGGACTTCAGGCCCTAT GGGTCAAACGACCCTAG GGGACGTAAGTCCCTAA TGGATTACCGACCGCAG GTGACCGACGTCCCCAG AGGACCTTCGGCCCCAC GGGACTTCTGTCCCTAG GGGACTTTCGGCCCTGT GGGACCAACGCCCCTGG GGGACCGAAGTCCCCGG 43
Per-base score for k = 17 is: 2.5294117647058822
GGGACTTCAGGCCCTATC GGGTCAAACGACCCTAGT GGGACGTAAGTCCCTAAC GCCGTCTCAGTACCCAGC GTGACCGACGTCCCCAGC AGGACCTTCGGCCCCACC GGGACTTCTGTCCCTAGC GGGACTTTCGGCCCTGTC GGGACCAACGCCCCTGGG GGGACCGAAGTCCCCGGG 46
Per-base score for k = 18 is: 2.5555555555555554
GGGACTTCAGGCCCTATCG GGGTCAAACGACCCTAGTG GGGACGTAAGTCCCTAACG GCCGTCTCAGTACCCAGCC GTGACCGACGTCCCCAGCC AGGAC

## Zero in on 20-mer motifs, and find the consensus motif

Run a longer search on the data for a 20-mer motif matrix

In [ ]:
score, motifMatrix = gibbsSamplerScript.runGibbsSampler(dna=dna, k=20, t=t, reps=50, searchLength=2000)
consensus = medianString.medianString(motifMatrix, 20)

## Find associated coding regions

Now that we have a good idea of the motif, we can find the index of the motif in each string